In [1]:
# -*- coding: utf-8 -*-
import string
import urllib 
import json
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
import random

def set_up_parameters():
    cruiseLines=["Carnival","Celebrity", "Costa", "Disney","Msc", "Norvegian", "Princess", 
                "P&O", "Royal Caribbean"]  
    myStopWords=["more","less","would","should","read","review","carnival","celebrity", "costa", "disney","msc", 
                 "norvegian", "princess", "p&o", "royal caribbean"]
    cruiseKinds=["senior cruises", "romantic cruises","family cruises", "first-time cruises"]
    cruiseKindUrls={"family cruises":"-family-cruises", 
                    "first-time cruises":"-first-time-cruiser-cruises",
             "senior cruises":"-senior-cruises", "romantic cruises":"-romantic-cruises"}    
    cruiseKindDesc={"family cruises":"Histogram: Family Cruises", 
                    "first-time cruises":"Histogram: First-Time Cruisers",
             "senior cruises":"Histogram: First-Time Cruisers", "romantic cruises":"Histogram: Romantic Cruises"}    
    cruiseLineId={"Msc":'80', "Carnival":'8',"Costa": '45', "Celebrity": '9', "Disney": '16', 
                  "Norvegian": '24', "Princess": '28', "P&O": '81', 
                  "Royal Caribbean": '32'}    
    cruiseLineNames={"Msc":'msc-cruises', "Carnival":'carnival',"Costa": 'costa', 
                     "Celebrity": 'celebrity', "Disney": 'disney', 
                  "Norvegian": 'norwegian-ncl', "Princess": 'princess', 
                  "P&O": 'pando',  "Royal Caribbean": 'royal-caribbean'} 
    cruiseLineUrls={"Msc":'msc-cruises', "Carnival":'carnival-cruises',"Costa": 'costa-cruises', 
                     "Celebrity": 'celebrity-cruises', "Disney": 'disney-cruises', 
                  "Norvegian": 'norwegian-ncl-cruises', "Princess": 'princess-cruises', 
                  "P&O": 'pando-cruises',  "Royal Caribbean": 'royal-caribbean-cruises'}    
    sailMonths= {
                "January 2008":"08-01", "February 2008":"08-02","March 2008":"08-03",
                "April 2008":"08-04", "May 2008":"08-05","June 2008":"08-06",
                "July 2008":"08-07", "August 2008":"08-08","September 2008":"08-09",
             "October 2008":"08-10","November 2008":"08-11","December 2008":"08-12",
                "January 2009":"09-01", "February 2009":"09-02","March 2009":"09-03",
                "April 2009":"09-04", "May 2009":"09-05","June 2009":"09-06",
                "July 2009":"09-07", "August 2009":"09-08","September 2009":"09-09",
             "October 2009":"09-10","November 2009":"09-11","December 2009":"09-12",
                "January 2010":"10-01", "February 2010":"10-02","March 2010":"10-03",
                "April 2010":"10-04", "May 2010":"10-05","June 2010":"10-06",
                "July 2010":"10-07", "August 2010":"10-08","September 2010":"10-09",
             "October 2010":"10-10","November 2010":"10-11","December 2010":"10-12",
                "January 2011":"11-01", "February 2011":"11-02","March 2011":"11-03",
                "April 2011":"11-04", "May 2011":"11-05","June 2011":"11-06",
                "July 2011":"11-07", "August 2011":"11-08","September 2011":"11-09",
             "October 2011":"11-10","November 2011":"11-11","December 2011":"11-12",
                "January 2012":"12-01", "February 2012":"12-02","March 2012":"12-03",
                "April 2012":"12-04", "May 2012":"12-05","June 2012":"12-06",
                "July 2012":"12-07", "August 2012":"12-08","September 2012":"12-09",
             "October 2012":"12-10","November 2012":"12-11","December 2012":"12-12",
                "January 2013":"13-01", "February 2013":"13-02","March 2013":"13-03",
                "April 2013":"13-04", "May 2013":"13-05","June 2013":"13-06",
                "July 2013":"13-07", "August 2013":"13-08","September 2013":"13-09",
             "October 2013":"13-10","November 2013":"13-11","December 2013":"13-12",
                "January 2014":"14-01", "February 2014":"14-02","March 2014":"14-03",
                "April 2014":"14-04", "May 2014":"14-05","June 2014":"14-06",
                "July 2014":"14-07", "August 2014":"14-08","September 2014":"14-09",
             "October 2014":"14-10","November 2014":"14-11","December 2014":"14-12",
             "January 2015":"15-01","February 2015":"15-02"
             }
    
    sailYears={"2008":"2008", "2009":"2009","2010":"2010","2011":"2011","2012":"2012","2013":"2013","2014":"2014","2015":"2015"}
    
    return myStopWords, cruiseLines, cruiseKinds, cruiseKindUrls, cruiseKindDesc,cruiseLineId, cruiseLineNames, sailMonths,cruiseLineUrls

myStopWords, cruiseLines, cruiseKinds, cruiseKindUrls, cruiseKindDesc,cruiseLineId, cruiseLineNames, sailMonths,cruiseLineUrls = set_up_parameters()

def open_url(link):
    sock = urllib.urlopen(link)
    htmlSource = sock.read()
    sock.close() 
    return htmlSource

def initialize_beautifulSoup(htmlSource):
    htmlSource = convert_encoding(htmlSource)
    htmlSource = BeautifulSoup(htmlSource)
    return htmlSource

def convert_encoding(htmlSource):
    return htmlSource

def find_number_of_comments_per_kind(htmlSource,cruiseKind,cruiseLineLink):
    """
    returns the number of reviews of a given line, and of a given cruise kind
    """
    findTag="div"
    findId="member-reviews-histogram-cruise-reviews-container"
    findAttrTitle="data-sname"
    findAttrContent=cruiseKindDesc[cruiseKind]
    found=htmlSource.find_all(findTag, id=findId)
    numberOfComments=""
    for element in found:
        foundTwo=element.findAll(attrs={findAttrTitle: findAttrContent})
        for elementTwo in foundTwo:
            foundThree=elementTwo.findAll(findTag, class_="pull-right")
            for numberOfComments in foundThree:
                numberOfComments=numberOfComments.text
    replaceStrings=[".",","]
    for character in replaceStrings:
        numberOfComments = numberOfComments.replace(character, "")
    if numberOfComments=="":
        print "error getting number of comments from",cruiseLineLink
    else:
        numberOfComments=int(numberOfComments)
    return numberOfComments
        
def find_first_comment_line(htmlSource,cruiseLineLink):
    """
    returns first commentid
    """
    findTag="div"     
    found=htmlSource.find_all(findTag, class_="clearfix review-results-container")
    result=[]
    for element in found:
        foundTwo=element.findAll(findTag, class_="clearfix review-results-item-container border-bottom-grey")
        for elementTwo in foundTwo:
            foundThree=elementTwo.findAll(findTag, class_="clearfix")
            for elementThree in foundThree:
                foundFour=elementThree.findAll(findTag, class_="pull-left review-results-title")
                for elementFour in foundFour:
                    foundFive=elementFour.find_all('a')
                    for elementFive in foundFive:
                        result.append(elementFive.get("href")[40:46]) 
    if result == []:
        print "error getting commentId in ", cruiseLineLink
    else:
        result = str(result[0])    
    return result


def find_first_comment_ship(htmlSource,cruiseShipLink):
    findTag="div"     
    found=htmlSource.find_all(findTag, class_="clearfix review-results-container")
    result=[]
    for element in found:
        foundTwo=element.findAll(findTag, class_="clearfix review-results-item-container border-bottom-grey")
        for elementTwo in foundTwo:
            foundThree=elementTwo.findAll(findTag, class_="pull-left review-results-title")
            for elementThree in foundThree:
                foundFour=elementThree.find_all('a')
                for elementFour in foundFour:
                    result.append(elementFour.get("href")[40:])
    if result == []:
        print "error getting commentId in ", cruiseShipLink
    else:
        return str(result[0])    

def find_first_comment_per_ship(shipIds):
    shipComments={}
    for shipId in shipIds:
        cruiseShipLink="http://www.cruisecritic.com/memberreviews/getreviews.cfm?action=ship&ShipID="+shipId 
        htmlSource = open_url(cruiseShipLink)
        htmlSource =initialize_beautifulSoup(htmlSource)
        commentId=find_first_comment_ship(htmlSource,cruiseShipLink)
        shipComments[shipId]=commentId
    return shipComments

def find_sail_dates(htmlSource,fullLink):      
    findTag="div" 
    findId="review-details"
    found=htmlSource.find_all(findTag, id=findId)
    sailDate=""
    for element in found:
        foundTwo=element.findAll(findTag, class_="col-sm-4")
        for elementTwo in foundTwo:
            if "Sail Date" in str(elementTwo.contents):
                elementThree = elementTwo.contents
                sailDate = elementThree[2].strip()
    if sailDate=="":
        print "error getting sailDate in", fullLink
        sailDate="not specified"
    else:
        return sailDate

def filterout_old_dates(sailDate):
    if sailDate in sailMonths.keys():
        sailMonth=sailMonths[sailDate]
        return sailMonth
    else:
        return False

def find_ship(htmlSource,fullLink): 
    ship=""
    findTag="div" 
    findId="check-prices-div"
    found=htmlSource.find_all(findTag, id=findId)
    for element in found:  
        foundTwo=element.findAll('button',{"data-ship-name":True})
        for elementTwo in foundTwo:
            ship= elementTwo['data-ship-name']
    if ship == "":
        print "error getting ship info from", fullLink
        ship="not specified"
        return ship
    else:
        return ship

def find_rating(htmlSource): 
    findTag="div" 
    findId="rating-info"
    rating="no rating"
    found=htmlSource.find_all(findTag, id=findId)
    for foundImg in found:
        imgTag=foundImg.findAll("img")
        for char in str(imgTag):
            if char.isdigit():
                rating = char
    otherRatings = find_sub_ratings(found)
    return rating, otherRatings 
                
def find_sub_ratings(found):                
    otherRatings={}
    for element in found:
        table = element.findAll('tr')
        count = 0
        ratingSum=0
        for elementTwo in table:
            row = elementTwo.findAll("td")
            if row[1].text != "Not Rated":
                otherRatings[row[0].text]=row[1].text
                count=count+1.0
                ratingSum = ratingSum + float(row[1].text)            
        if ratingSum!=0:
            bestRated=', '.join(str(key) for max_value in (max(otherRatings.values()),) for key in otherRatings if otherRatings[key]==max_value)
            worseRated=', '.join(str(key) for min_value in (min(otherRatings.values()),) for key in otherRatings if otherRatings[key]==min_value)
            worseRated=worseRated.split(',')
            bestRated=bestRated.split(',')
            random.shuffle(worseRated) 
            random.shuffle(bestRated) 
            otherRatings["best_rating"]=bestRated[0]
            otherRatings["worse_rating"]=worseRated[0]
        else:
            otherRatings["best_rating"]="not specified"
            otherRatings["worse_rating"]="not specified"        
    return otherRatings 

def find_comment(htmlSource,fullLink):
    findTag="div" 
    findId="main-review"
    comment=""
    found=htmlSource.find_all(findTag, id=findId)
    for element in found:
        commentText=element.findAll("p")
        for text in commentText:
            comment = text.text           
    end=comment.find("Read more")
    comment=comment[:end]
    end=comment.find("Less")
    if comment=="":
        print "error getting review text from", fullLink
        return "",[],[]
    else:    
        comment=comment[:end]
        commentWords,lemmatizedWords=preprocess(comment)
        return comment,commentWords,lemmatizedWords

def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    filtered_words = [w for w in filtered_words if not w in myStopWords]
    comment = " ".join(filtered_words)
    commentWords=comment.split()
    commentWords = [w for w in commentWords if len(w) > 2] 
    commentWords = [w for w in commentWords if not w.isdigit()] 
    lemmatizer = WordNetLemmatizer()
    lemmatizedWords = [lemmatizer.lemmatize(t) for t in commentWords]    
    return commentWords, lemmatizedWords
       
def find_commentId(htmlSource,fullLink): 
    findTag="div" 
    findId="bottom-nav-buttons"
    commentId=""
    found=htmlSource.find_all(findTag, id=findId)
    for element in found:
        foundTwo=element.findAll(findTag, class_="pull-right")
        for elementTwo in foundTwo:
                    foundThree=elementTwo.find_all('a')
                    for elementThree in foundThree:
                        commentId = elementThree.get("href")[40:]
    return commentId

def get_first_comment_id_and_numerosity_per_kind(cruiseLine, cruiseKind):
    cruiseLineLink="http://www.cruisecritic.com/"+cruiseLineNames[cruiseLine]+cruiseKindUrls[cruiseKind]+"/cst/" 
    htmlSource = open_url(cruiseLineLink)
    htmlSource =initialize_beautifulSoup(htmlSource)
    numberOfCommentsPerKind=find_number_of_comments_per_kind(htmlSource,cruiseKind,cruiseLineLink)
    print "found",numberOfCommentsPerKind,'for', cruiseKind, 'cruises','for', cruiseLine
    commentId=find_first_comment_line(htmlSource,cruiseLineLink)
    return commentId,numberOfCommentsPerKind


def get_Year(sailDate):
    sailYear=""
    for char in sailDate:
        if char.isdigit():
            sailYear=sailYear+char      
    return str(sailYear)

def get_comment_text_and_new_comment(commentId, mydict,line,kind): 
    commentLink="http://www.cruisecritic.com/memberreviews/memberreview.cfm?EntryID="
    fullLink=commentLink+commentId
    htmlSource = open_url(fullLink)
    htmlSource =initialize_beautifulSoup(htmlSource)
    commentId = find_commentId(htmlSource,fullLink)   
    if commentId != "" and commentId.isdigit() and commentId not in mydict:
        sailDate = find_sail_dates(htmlSource,fullLink)
        sailMonth= filterout_old_dates(sailDate)
        sailYear=get_Year(sailDate)
        if not sailMonth:
            pass
        else:
            ship=find_ship(htmlSource,fullLink) 
            rating, otherRatings=find_rating(htmlSource)
            comment,commentWords,lemmatizedWords=find_comment(htmlSource,fullLink)
            mydict[commentId]={"comment":comment,"lemmatizedWords":lemmatizedWords,"ship":ship, 
                           "rating": rating, "kind": kind,
                           "sail Date": sailDate, "sail Month": sailMonth, "sail Year": sailYear, 
                           "source":"cruisecritic.com", "otherRatings":otherRatings}  
    return mydict,commentId

def find_ship_ids_and_number_of_comments_per_ship(htmlSource,cruiseLineLink):
    shipIds={}
    findTag="div" 
    findClass="cruise-line-main-results-container clearfix"
    foundOne=htmlSource.find_all(findTag, class_=findClass)
    for element in foundOne:
        shipInfo=element.findAll("a")
        count = 1
        for shipLink in shipInfo:
            if count == 3:
                result=""
                shipId=shipLink.get("href")[49:]
                numberOfRatings=shipLink.text.strip()
                for char in numberOfRatings:
                    if char.isdigit():
                        result=result+char                
                shipIds[shipId]=int(result)
            count = count + 1
    if shipIds=={}:
        print "error getting shipIds in ", cruiseLineLink
    else:
        return shipIds

def get_first_comment_id_and_numerosity_per_ship(cruiseLine):
    cruiseLineLink="http://www.cruisecritic.com/memberreviews/"+cruiseLineUrls[cruiseLine]+"/cl/"
    htmlSource = open_url(cruiseLineLink)
    htmlSource =initialize_beautifulSoup(htmlSource)
    shipIds=find_ship_ids_and_number_of_comments_per_ship(htmlSource,cruiseLineLink)
    shipComments=find_first_comment_per_ship(shipIds)
    return shipComments, shipIds

        
def main(maximum):
    commentIds={}
    print "started processing"
    for line in cruiseLines:
        mydict=load_old_review_file(line)
        for kind in cruiseKinds:
            if len(mydict) < maximum:
                commentId, numberOfCommentsPerKind = get_first_comment_id_and_numerosity_per_kind(line, kind) 
                count = 0 
                while count <= numberOfCommentsPerKind and commentId.isdigit() and len(mydict) < maximum :
                    mydict, commentId = get_comment_text_and_new_comment(commentId, mydict, line, kind)
                    count = count + 1
                    if count % 100 == 0:
                        print 'finished getting', count, 'comment for', line
        shipComments, shipIds=get_first_comment_id_and_numerosity_per_ship(line)
        for shipId in shipIds:
            if len(mydict) < maximum:
                count = 0
                commentId=shipComments[shipId]
                numberOfCommentsPerShip=shipIds[shipId]
                while count <= numberOfCommentsPerShip and commentId.isdigit() and len(mydict) < maximum:
                    mydict, commentId = get_comment_text_and_new_comment(commentId, mydict, line, "")
                    count = count + 1
                    if count % 100 == 0:
                        print 'finished getting', count, 'comment for', line
        save_both(mydict,line)  
        print "finished processing", line
    print "finished all"
    return mydict

def save_both(mydict,line):
    suffix=""        
    save_to_json(mydict,line,suffix)
    suffix="_old"
    save_to_json(mydict, line,suffix)
    print 'saved',line,"new review json with",len(mydict),'reviews'
    

def load_old_review_file(line):
    count = 0
    with open('data/'+line+'_old.json', 'rb') as fp:
        mydict = json.load(fp)
        print 'loaded',line,"old review json with",len(mydict),'reviews'
    return mydict

def save_to_json(mydict, name, suffix):
    with open("data/"+name+suffix+'.json', 'wb') as fp:
        json.dump(mydict, fp)


set_up_parameters()              
mydict = main(2000)

started processing
loaded Royal Caribbean old review json with 0 reviews
found 1002 for senior cruises cruises for Royal Caribbean
finished getting 100 comment for Royal Caribbean
finished getting 200 comment for Royal Caribbean
finished getting 300 comment for Royal Caribbean
finished getting 400 comment for Royal Caribbean
error getting review text from http://www.cruisecritic.com/memberreviews/memberreview.cfm?EntryID=55304
finished getting 500 comment for Royal Caribbean
finished getting 600 comment for Royal Caribbean
finished getting 700 comment for Royal Caribbean
finished getting 800 comment for Royal Caribbean
found 894 for romantic cruises cruises for Royal Caribbean
finished getting 100 comment for Royal Caribbean
finished getting 200 comment for Royal Caribbean
finished getting 300 comment for Royal Caribbean
finished getting 400 comment for Royal Caribbean
finished getting 500 comment for Royal Caribbean
finished getting 600 comment for Royal Caribbean
finished getting 700